# 旅行商问题

## 目标和前提条件

在这个例子中，您将学习如何解决最著名的组合优化问题之一：旅行商问题（TSP）。TSP的目标很简单——找到最短的路线，访问每个城市一次并返回起始城市——但解决这个问题是一项复杂且具有挑战性的任务。我们将向您展示如何做到这一点！

这个建模示例属于高级水平，我们假设您了解Python和Gurobi Python API，并且具有构建数学优化模型的高级知识。通常，这些例子的目标函数和/或约束条件很复杂，或者需要Gurobi Python API的高级特性。

**下载仓库** <br />
您可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的仓库。

## 动机

旅行商问题（TSP）是最著名的组合优化问题之一。这个问题很容易解释，但非常难以解决——即使对于规模较小的城市实例也是如此。关于TSP的更详细信息可以在《旅行商问题：计算研究》[1]一书中找到，或者在TSP主页[2]上查看。如果您对TSP的历史和数学背景感兴趣，我们建议您观看William Cook的视频[3]。

旅行商问题的起源并不十分清晰；它在1832年的一本旅行推销员手册中被提及，其中包括45个德国城市的示例路线，但并未被表述为数学问题。然而，在19世纪，数学家William Rowan Hamilton和Thomas Kirkman提出了该问题的数学表述。

旅行商问题的一般形式似乎最早是由Karl Menger在20世纪30年代在维也纳和哈佛进行研究的。

这个问题在20世纪50年代和60年代变得越来越流行。特别是，兰德公司的George Dantzig、D. Ray Fulkerson和Selmer M. Johnson通过将其表述为线性规划问题解决了48个州的问题。他们在相关论文中描述的方法为未来组合优化领域的工作奠定了基础，特别强调了割平面的重要性。

在20世纪70年代初，P vs. NP问题的概念在理论计算机科学界引起了轰动。1972年，Richard Karp证明了哈密顿回路问题是NP完全的，这意味着旅行商问题是NP困难的。

越来越复杂的算法导致了可解决的旅行商问题规模的快速增长。Dantzig、Fulkerson和Johnson在1954年解决了一个48城市的实例。Martin Grötschel在23年后将这一数字增加了一倍多，在1977年解决了一个120城市的实例。Harlan Crowder和Manfred W. Padberg仅用3年时间再次将这一数字增加了一倍多，解决了一个318城市的问题。

1987年，算法取得了快速进步，culminating in a 2,392城市的解决方案，由Padberg和Giovanni Rinaldi提出。在接下来的20年里，取得了巨大进展，David L. Applegate、Robert E. Bixby、Vasek Chvátal和William J. Cook在1992年解决了3,308城市的实例，1994年解决了7,397城市的实例，2004年解决了24,978城市的实例，以及2006年解决了85,900城市的实例——这是有史以来解决的最大的2-D欧几里得TSP实例。William Cook等人编写了一个名为Concorde TSP Solver的程序来解决TSP[4]。Concorde是一个用于对称TSP和一些相关网络优化问题的计算机代码。该代码使用ANSI C编程语言编写，已被用于获得全部110个TSPLIB实例的最优解，最大的实例是一个109,399节点的3-D"星型"实例。

对TSP持续的兴趣可以通过其作为发现的通用引擎和源源不断的新应用来解释。TSP的一些通用应用如下：
* 调度和路径规划问题。
* 基因组测序。
* 钻孔问题。
* 望远镜和X射线的对准。
* 数据聚类。
* 机器调度。

我们使用这个经典的组合优化问题来演示Gurobi如何被用来轻松有效地解决小规模的TSP实例。然而，为了能够解决更大规模的实例，需要更复杂的技术——比如Concorde TSP Solver中实现的那些。

## 问题描述
TSP可以定义如下：对于给定的城市列表以及每对城市之间的距离，我们希望找到最短的路线，该路线访问每个城市一次并返回起始城市。

有一类旅行商问题假设从城市$i$到城市$j$的距离与从城市$j$到城市$i$的距离相同，这类旅行商问题也被称为对称旅行商问题。在本例中，我们使用欧几里得距离，但TSP模型的构建对于确定各个距离的方式是独立的。

## 解决方案方法

数学规划是一种声明式方法，其中建模者制定一个数学优化模型，捕捉复杂决策问题的关键方面。Gurobi优化器使用先进的数学和计算机科学技术解决这些模型。

数学优化模型有五个组成部分，即：

* 集合和索引。
* 参数。
* 决策变量。
* 目标函数。
* 约束条件。

现在我们提出一个TSP的MIP（混合整数规划）模型，它确定了访问所有城市一次并返回起始城市的最短路线。

## TSP模型构建

### 集合和索引
$i, j \in Capitals $：美国首府城市的索引和集合。

$\text{Pairings}= \{(i,j) \in Capitals \times Capitals \}$：允许的配对集合

$S \subset Capitals$：美国首府城市集合的子集。

$G = (Capitals, Pairings)$：一个图，其中集合$Capitals$定义节点集，集合$Pairings$定义边集。

### 参数

$d_{i, j} \in \mathbb{R}^+$：从首府城市$i$到首府城市$j$的距离，对于所有$(i, j) \in Pairings$。

注意，从首府城市$i$到首府城市$j$的距离与从首府城市$j$到首府城市$i$的距离相同，即$d_{i, j} = d_{j, i}$。因此，这个TSP也被称为对称旅行商问题。

### 决策变量
$x_{i, j} \in \{0, 1\}$：如果我们决定连接城市$i$和城市$j$，则该变量等于1。否则，决策变量等于零。

### 目标函数
- **最短路线**。最小化路线的总距离。路线是一系列首府城市，其中推销员只访问每个城市一次，并返回起始首府城市。

\begin{equation}
\text{Min} \quad Z = \sum_{(i,j) \in \text{Pairings}}d_{i,j} \cdot x_{i,j}
\tag{0}
\end{equation}

### 约束条件
- **对称约束**。对于每条边$(i,j)$，确保首府城市$i$和$j$相连，如果前者是在访问后者之前或之后立即访问的。

\begin{equation}
x_{i, j} = x_{j, i} \quad \forall (i, j) \in Pairings
\tag{1}
\end{equation}

- **进入和离开首府城市**。对于每个首府城市$i$，确保该城市与另外两个城市相连。

\begin{equation}
\sum_{(i,j) \in \text{Pairings}}x_{i,j} = 2 \quad \forall  i \in Capitals
\tag{2}
\end{equation}

- **子回路消除**。这些约束确保对于$Capitals$集合的任何子集$S$，都没有循环。也就是说，没有路线访问子集中的所有城市并返回起始城市。

\begin{equation}
\sum_{(i \neq j) \in S}x_{i,j} \leq |S|-1 \quad \forall  S \subset  Capitals
\tag{3}
\end{equation}

- **备注**。一般来说，如果TSP中的城市数量是$n$，那么可能的路线数量是n\!。
由于消除循环的约束数量呈指数级增长($2^{n} - 2$)，我们使用懒约束来动态消除这些循环。

## Python实现

考虑一个需要拜访美国大陆各州首府客户的推销员。推销员希望确定访问所有州首府的最短路线。

此建模示例需要以下不属于标准Python发行版的库：
* **folium**：用于创建地图。
* **gurobipy**：提供Gurobi算法来解决MIP模型。


### 读取输入数据
首府名称和坐标从json文件中读取。

In [ ]:
%pip install gurobipy

In [ ]:
import json

# 从json文件读取首府名称和坐标
try:
  capitals_json = json.load(open('capitals.json'))
# 在本地运行时可以省略以下几行
except: 
  import urllib.request
  url = 'https://raw.githubusercontent.com/Gurobi/modeling-examples/master/traveling_salesman/capitals.json'
  data = urllib.request.urlopen(url).read()
  capitals_json = json.loads(data)

capitals = []
coordinates = {}
for state in capitals_json:
    if state not in ['AK', 'HI']:
      capital = capitals_json[state]['capital']
      capitals.append(capital)
      coordinates[capital] = (float(capitals_json[state]['lat']), float(capitals_json[state]['long']))

### 数据计算
以下函数计算每对州首府之间的距离。由于我们正在解决_对称_旅行商问题，我们使用城市的_组合_。

In [ ]:
import math
from itertools import combinations

# 计算成对距离矩阵

def distance(city1, city2):
    c1 = coordinates[city1]
    c2 = coordinates[city2]
    diff = (c1[0]-c2[0], c1[1]-c2[1])
    return math.sqrt(diff[0]*diff[0]+diff[1]*diff[1])

dist = {(c1, c2): distance(c1, c2) for c1, c2 in combinations(capitals, 2)}

### 模型代码
现在我们通过定义决策变量、约束条件和目标函数来编写TSP模型。因为这是_对称_旅行商问题，我们可以通过将_对象_x[j,i]设置为x[i,j]来提高效率，而不是使用约束条件。

In [ ]:
import gurobipy as gp
from gurobipy import GRB

# 使用Python 3.7和Gurobi 9.0.0测试

m = gp.Model()

# 变量：城市'i'是否在路径上与城市'j'相邻？
vars = m.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='x')

# 对称方向：使用dict.update为变量设置新键的别名
vars.update({(j,i):vars[i,j] for i,j in vars.keys()})

# 约束：每个城市都有两条边连接
cons = m.addConstrs(vars.sum(c, '*') == 2 for c in capitals)

### 回调定义
子回路约束防止TSP路线中出现多个循环。由于这些约束的数量是指数级的，我们不想将它们全部添加到模型中。相反，我们使用回调函数来查找被违反的子回路约束，并将它们作为懒约束添加到模型中。

In [ ]:
# 回调 - 使用懒约束消除子回路

def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # 创建解决方案中选中的边的列表
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i, j) for i, j in model._vars.keys()
                             if vals[i, j] > 0.5)
        # 在选定的边列表中找到最短的循环
        tour = subtour(selected)
        if len(tour) < len(capitals):
            # 为子回路中的每对城市添加子回路消除约束
            model.cbLazy(gp.quicksum(model._vars[i, j] for i, j in combinations(tour, 2))
                         <= len(tour)-1)

# 给定边的元组列表，找到最短的子回路

def subtour(edges):
    unvisited = capitals[:]
    cycle = capitals[:] # 虚拟值 - 保证会被替换
    while unvisited:  # 如果列表非空则为真
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # 新的最短子回路
    return cycle

## 求解模型

In [ ]:
m._vars = vars
m.Params.lazyConstraints = 1
m.optimize(subtourelim)

## 分析

我们检索TSP的最优解，并验证最优路线（或巡回路线）访问了所有城市并返回到起始城市。

In [ ]:
# 检索解决方案

vals = m.getAttr('x', vars)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

tour = subtour(selected)
assert len(tour) == len(capitals)

最优路线在下面的地图中显示。

In [ ]:
# 将解决方案映射到地图上

import folium

map = folium.Map(location=[40,-95], zoom_start = 4)

points = []
for city in tour:
  points.append(coordinates[city])
points.append(points[0])

folium.PolyLine(points).add_to(map)

map

In [ ]:
m.dispose()
gp.disposeDefaultEnv()

## 结论

旅行商问题（TSP）是最流行的组合优化问题。这个问题很容易解释，尽管解决起来非常复杂。已解决的最大TSP问题有85,900个城市。TSP是发现解决复杂组合优化问题的新方法的源泉，并已导致许多应用。

在这个建模示例中，我们展示了如何将对称旅行商问题构建为MIP问题。我们还展示了如何使用懒约束动态消除子回路。

## 参考文献

[1] D. L. Applegate, R. E. Bixby, V. Chvatal and W. J. Cook , The Traveling Salesman Problem: A Computational Study, Princeton University Press, Princeton, 2006.

[2] http://www.math.uwaterloo.ca/tsp/index.html

[3] https://www.youtube.com/watch?v=q8nQTNvCrjE&t=35s

[4] http://www.math.uwaterloo.ca/tsp/concorde.html

Copyright © 2020 Gurobi Optimization, LLC